# Installation
From [the docs](http://qcodes.github.io/Qcodes/start/index.html):

1. Install [Anaconda](https://www.anaconda.com/download/). Get the Python 3 version. No need to install the VSCode stuff.
1. Save [environment.yml](https://raw.githubusercontent.com/QCoDeS/Qcodes/master/environment.yml) to the machine.
1. Run the following in an Anaconda Prompt:
```
conda env create -f environment.yml
activate qcodes
pip install qcodes
```
1. Now you can run `jupyter notebook` from within your qcodes environment.

In [ ]:
import time
import numpy as np
import qcodes as qc
from qcodes.dataset.measurements import Measurement

# Instrument setup

At this point, each measurement setup needs to define which instruments they use. See [the docs](http://qcodes.github.io/Qcodes/user/intro.html#instrument). For demonstration purposes, here I just use dummy instruments `dac` and `dmm`.

In [ ]:
from qcodes.tests.instrument_mocks import DummyInstrument
dac = DummyInstrument(name="dac", gates=['ch1', 'ch2'])
dmm = DummyInstrument(name="dmm", gates=['idc', 'ig'])

import random
dmm.idc.get = lambda: random.gauss(1, 0.1)
dmm.ig.get = lambda: random.gauss(0, 0.01)

You can set `step` and `inter_delay` on any parameters. These will apply to all future attempts to set the value, including those in loops. For instance, if `dac.ch1` can be ramped at a maximum speed of 0.1 V / ms, then do this:

In [ ]:
dac.ch1.step = 0.1
dac.ch1.inter_delay = 0.001

# Experiment setup

The first time you run qcodes on a new computer, you need to create the SQLite database. See [the docs](https://qcodes.github.io/Qcodes/dataset/index.html) for details on the design of the dataset and database. You will then want to make a new experiment to hold your data. At the very least, make a new experiment when you change out your samples.

In [ ]:
qc.initialise_database()
qc.new_experiment(name='demo', sample_name='my best sample')

# Measurement

## 1D sweep

In its most basic form, a 1D sweep looks like this. Below, I will go into more details.

In [ ]:
meas = Measurement()
meas.register_custom_parameter('x')
meas.register_custom_parameter('y', setpoints=('x',))

with meas.run() as datasaver:
    for x in np.linspace(0, 25, 10):
        # Set x.
        # Measure y.
        datasaver.add_result(('x', x), ('y', 0.0))

We use a `Measurement` object to specify which parameters are dependent and which are independent. In this example, I define two of each. Note that the `'time'` parameter is not a `qcodes.Parameter`!

In [ ]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_custom_parameter('time', label='Time', unit='s')
meas.register_parameter(dmm.ig, setpoints=(dac.ch1, 'time',))
meas.register_parameter(dmm.idc, setpoints=(dac.ch1, 'time',))

A measurement's `write_period` specifies how often to write to the database, in seconds. This also affects how often to plot.

In [ ]:
meas.write_period = 1

Add actions and after the run as necessary. This is useful for things like sweeping up gates, ensuring the magnet is off after the run, and so on.

In [ ]:
meas.add_before_run(lambda dac: dac.ch2.set(10), (dac,))
meas.add_after_run(lambda dac: dac.ch2.set(0), (dac,))

`meas.run()` will give you a nice context manager which you can use to save data. I like to catch `KeyboardInterrupt`, which will be sent when the stop button is pressed in jupyter.

In [ ]:
with meas.run() as datasaver:
    try:
        time.sleep(5)
        t0 = time.monotonic()
        for set_v in np.linspace(0, 25, 10):
            dac.ch1.set(set_v)
            time.sleep(1)
            datasaver.add_result(
                (dac.ch1, set_v),
                (dmm.ig, dmm.ig.get()),
                (dmm.idc, dmm.idc.get()),
                ('time', time.monotonic() - t0))
    except KeyboardInterrupt:
        pass

I've encapsulated 1D sweeping and plotting into a Sweep1D class. It will autorange SR830 lockins too!

In [ ]:
# %load sweep.py
import io
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import qcodes as qc
from qcodes.dataset.measurements import Measurement
from IPython import display

def _autorange_srs(srs, max_changes=1):
    def autorange_once():
        r = srs.R.get()
        sens = srs.sensitivity.get()
        if r > 0.9 * sens:
            return srs.increment_sensitivity()
        elif r < 0.1 * sens:
            return srs.decrement_sensitivity()
        return False
    sets = 0
    while autorange_once() and sets < max_changes:
        sets += 1
        time.sleep(10*srs.time_constant.get())

class Sweep(object):
    def __init__(self):
        self._sr830s = []
        self._params = []
    
    def follow_param(self, p):
        self._params.append(p)

    def follow_sr830(self, l, name, gain=1.0):
        self._sr830s.append((l, name, gain))

    def _create_measurement(self, *set_params):
        meas = Measurement()
        for p in set_params:
            meas.register_parameter(p)
        meas.register_custom_parameter('time', label='Time', unit='s')
        for p in self._params:
            meas.register_parameter(p, setpoints=(*set_params, 'time',))
        for l, _, _ in self._sr830s:
            meas.register_parameter(l.X, setpoints=(*set_params, 'time',))
            meas.register_parameter(l.Y, setpoints=(*set_params, 'time',))
        return meas
    
    def sweep(self, set_param, vals, inter_delay=None):
        if inter_delay is not None:
            d = len(vals)*inter_delay
            h, m, s = int(d/3600), int(d/60) % 60, int(d) % 60
            print(f'Minimum duration: {h}h {m}m {s}s')

        fig = plt.figure(figsize=(4*(2 + len(self._params) + len(self._sr830s)),4))
        grid = plt.GridSpec(4, 1 + len(self._params) + len(self._sr830s), hspace=0)
        setax = fig.add_subplot(grid[:, 0])
        setax.set_xlabel('Time (s)')
        setax.set_ylabel(f'{set_param.label} ({set_param.unit})')
        setaxline = setax.plot([], [])[0]

        paxs = []
        plines = []
        for i, p in enumerate(self._params):
            ax = fig.add_subplot(grid[:, 1 + i])
            ax.set_xlabel(f'{set_param.label} ({set_param.unit})')
            ax.set_ylabel(f'{p.label} ({p.unit})')
            paxs.append(ax)
            plines.append(ax.plot([], [])[0])

        laxs = []
        llines = []
        for i, (l, name, _) in enumerate(self._sr830s):
            ax0 = fig.add_subplot(grid[:-1, 1 + len(self._params) + i])
            ax0.set_ylabel(f'{name} (V)')
            fmt = ScalarFormatter()
            fmt.set_powerlimits((-3, 3))
            ax0.get_yaxis().set_major_formatter(fmt)
            laxs.append(ax0)
            llines.append(ax0.plot([], [])[0])
            ax1 = fig.add_subplot(grid[-1, 1 + len(self._params) + i], sharex=ax0)
            ax1.set_ylabel('Phase (°)')
            ax1.set_xlabel(f'{set_param.label} ({set_param.unit})')
            laxs.append(ax1)
            llines.append(ax1.plot([], [])[0])
            plt.setp(ax0.get_xticklabels(), visible=False)

        fig.tight_layout()
        fig.show()

        meas = self._create_measurement(set_param)
        with meas.run() as datasaver:
            t0 = time.monotonic()
            for setpoint in vals:
                t = time.monotonic() - t0
                set_param.set(setpoint)
                
                setaxline.set_xdata(np.append(setaxline.get_xdata(), t))
                setaxline.set_ydata(np.append(setaxline.get_ydata(), setpoint))
                setax.relim()
                setax.autoscale_view()
                
                if inter_delay is not None:
                    plt.pause(inter_delay)

                data = [
                    (set_param, setpoint),
                    ('time', t)
                ]
                for i, p in enumerate(self._params):
                    v = p.get()
                    data.append((p, v))
                    plines[i].set_xdata(np.append(plines[i].get_xdata(), setpoint))
                    plines[i].set_ydata(np.append(plines[i].get_ydata(), v))
                    paxs[i].relim()
                    paxs[i].autoscale_view()
                for i, (l, _, gain) in enumerate(self._sr830s):
                    _autorange_srs(l, 3)
                    x, y = l.snap('x', 'y')
                    x, y = x / gain, y / gain
                    data.extend([(l.X, x), (l.Y, y)])
                    llines[i*2].set_xdata(np.append(llines[i*2].get_xdata(), setpoint))
                    llines[i*2].set_ydata(np.append(llines[i*2].get_ydata(), x))
                    llines[i*2+1].set_xdata(np.append(llines[i*2+1].get_xdata(), setpoint))
                    llines[i*2+1].set_ydata(np.append(llines[i*2+1].get_ydata(), np.arctan2(y, x) * 180 / np.pi))
                    laxs[i*2].relim()
                    laxs[i*2].autoscale_view()
                    laxs[i*2+1].relim()
                    laxs[i*2+1].autoscale_view()

                datasaver.add_result(*data)
                
                fig.tight_layout()
                fig.canvas.draw()
                plt.pause(0.001)

            d = time.monotonic() - t0
            h, m, s = int(d/3600), int(d/60) % 60, int(d) % 60
            print(f'Completed in: {h}h {m}m {s}s')

            b = io.BytesIO()
            fig.savefig(b, format='png')
            display.display(display.Image(data=b.getbuffer(), format='png'))


In [ ]:
s = Sweep()
s.follow_param(dmm.ig)
s.follow_param(dmm.idc)
s.sweep(dac.ch1, np.linspace(0, 25, 10), inter_delay=0.5)
s.sweep(dac.ch1, np.linspace(25, 0, 10), inter_delay=0.5)
s.sweep(dac.ch1, np.concatenate((np.linspace(0, 25, 10), np.linspace(25, 0, 10))), inter_delay=0.1)

## 2D sweep

In [ ]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.ch2)
meas.register_parameter(dmm.ig, setpoints=(dac.ch1, dac.ch2))

with meas.run() as datasaver:
    try:
        for set_ch1 in np.linspace(0, 25, 10):
            for set_ch2 in np.linspace(0, 10, 10):
                dac.ch1.set(set_ch1)
                dac.ch2.set(set_ch2)
                ig = dmm.ig.get()

                datasaver.add_result((dac.ch1, set_ch1), (dac.ch2, set_ch2), (dmm.ig, ig))
    except KeyboardInterrupt:
        pass

## Where is the data?

See the `get_data_by_id` docstring.

TODO: Make this easier to save to a csv or something.

In [ ]:
get_data_by_id(datasaver.run_id)